# Pandas version of the code in "R for Marketing Research and Analytics"

## Ch 3 - Simulating data and plotting

In [13]:
from IPython.core.interactiveshell import InteractiveShell

# pretty print only the last output of the cell
#InteractiveShell.ast_node_interactivity = "last_expr"
# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"

In [14]:
import pandas as pd
import numpy as np

In [15]:
#define constants
k_stores = 20  # 20 stores, using "k_" for "constant"
k_weeks = 104   # 2 years of data each

In [16]:
# create data frame of initially missing values to hold the data
df_store = pd.DataFrame(data=np.empty(shape=(k_stores*k_weeks, 10)),
                        columns=['storNum', 'Year', 'Week', 'p1sales', 'p2sales', 'p1price','p2price','p1prom','p2prom','country'])
df_store

,storNum,Year,Week,p1sales,p2sales,p1price,p2price,p1prom,p2prom,country
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_store.shape

(2080, 10)

Create 2 series that will represent the store number and country for each observation

In [21]:
store_num = [str(num) for num in range(101,100+k_stores+1)] #make this string rather than int so mult works later
store_num
store_cty = ["US"]*3 + ["DE"]*5 + ["GB"]*3 + ["BR"]*2 + ["JP"]*4 + ["AU"]*1 + ["CN"]*2
store_cty
len(store_cty)  # make sure this is the right length

['101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120']

['US',
 'US',
 'US',
 'DE',
 'DE',
 'DE',
 'DE',
 'DE',
 'GB',
 'GB',
 'GB',
 'BR',
 'BR',
 'JP',
 'JP',
 'JP',
 'JP',
 'AU',
 'CN',
 'CN']

20

In [22]:
np.repeat(store_num, 3) #not what we're looking for but interesting in the future perhaps

array(['101', '101', '101', '102', '102', '102', '103', '103', '103',
       '104', '104', '104', '105', '105', '105', '106', '106', '106',
       '107', '107', '107', '108', '108', '108', '109', '109', '109',
       '110', '110', '110', '111', '111', '111', '112', '112', '112',
       '113', '113', '113', '114', '114', '114', '115', '115', '115',
       '116', '116', '116', '117', '117', '117', '118', '118', '118',
       '119', '119', '119', '120', '120', '120'], dtype='<U3')

In [23]:
# now replace the appropriate column in the dataframe with those values

# DAY TODO
#   These need to fill in to the right number of rows by repeating the 20 values x times
#df_store.storNum = np.repeat(store_num, k_weeks)
df_store.storNum = store_num * k_weeks
df_store.country = store_cty * k_weeks
df_store

,storNum,Year,Week,p1sales,p2sales,p1price,p2price,p1prom,p2prom,country
0,101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,US
1,102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,US
2,103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,US
3,104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DE
4,105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DE
5,106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DE
6,107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DE
7,108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DE
8,109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GB
9,110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GB


In [11]:
'A B C'.split()
'A B C'.split() * 6
'1 2 3'.split()
'1 2 3'.split() * 6

['A', 'B', 'C']

['A',
 'B',
 'C',
 'A',
 'B',
 'C',
 'A',
 'B',
 'C',
 'A',
 'B',
 'C',
 'A',
 'B',
 'C',
 'A',
 'B',
 'C']

['1', '2', '3']

['1',
 '2',
 '3',
 '1',
 '2',
 '3',
 '1',
 '2',
 '3',
 '1',
 '2',
 '3',
 '1',
 '2',
 '3',
 '1',
 '2',
 '3']